In [1]:
import glob
from pathlib import Path
from visual_utils import open3d_vis_utils as V
OPEN3D_FLAG = True
import numpy as np
import torch
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils
import struct
import pickle as pkl
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=False, root_path=None, logger=None, ext='.bin'):
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.root_path = root_path
        self.ext = ext
        data_file_list = glob.glob(str(root_path / f'*{self.ext}')) if self.root_path.is_dir() else [self.root_path]
        self.sample_file_list = data_file_list
    def __len__(self):
        return len(self.sample_file_list)

    def __getitem__(self, index):

        size_float = 4
        list_pcd = []
        with open(f'{self.root_path}/{index+1}.bin', "rb") as f:
            byte = f.read(size_float * 4)
            while byte:
                x, y, z, intensity = struct.unpack("ffff", byte)
                list_pcd.append([x, y, z, intensity])
                byte = f.read(size_float * 4)

        points = np.asarray(list_pcd)
        intensities = points[:, 3]
        min_intensity = np.min(intensities)
        max_intensity = np.max(intensities)
        normalized_intensities = (intensities - min_intensity) / (max_intensity - min_intensity)
        points[:, 3] = normalized_intensities

        input_dict = {
            'points': points,
            'frame_id': index+1,
        }
        data_dict = self.prepare_data(data_dict=input_dict)
        return data_dict

In [3]:
data_path = '/mnt/nas/users/shreyas21563/bin'
cfg_file = 'cfgs/argo2_models/cbgs_voxel01_voxelnext.yaml'
ckpt = '/home/dhvanil21040/OpenPCDet/VoxelNeXt_Argo2.pth'

In [4]:
cfg_from_yaml_file(cfg_file, cfg)
demo_dataset = DemoDataset(
        dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
        root_path=Path(data_path)
    )

In [5]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)

/home/dhvanil21040/miniconda3/envs/alive/lib/python3.7/site-packages/torch/cuda/__init__.py:143: UserWarning: 
NVIDIA GeForce RTX 4090 with CUDA capability sm_89 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 4090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [6]:
logger = common_utils.create_logger()
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=True)
model.cuda()
model.eval()
print("")

2024-10-30 22:26:13,673   INFO  ==> Loading parameters from checkpoint /home/dhvanil21040/OpenPCDet/VoxelNeXt_Argo2.pth to CPU


KeyError: 'model_state'

In [31]:
with torch.no_grad():
    for idx in tqdm(range(len(demo_dataset))):
        data_dict = demo_dataset[idx]
        data_dict = demo_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
        pred_dicts, _ = model.forward(data_dict)
        ref_boxes=pred_dicts[0]['pred_boxes'].cpu().numpy()
        ref_scores=pred_dicts[0]['pred_scores'].cpu().numpy()
        ref_labels=pred_dicts[0]['pred_labels'].cpu().numpy()
        predictions = {
            "ref_boxes": ref_boxes,
            "ref_scores": ref_scores,
            "ref_labels": ref_labels
        }
        with open(f"/mnt/nas/users/shreyas21563/predictions/{idx+1}.pkl", 'wb') as f:
            pkl.dump(predictions, f)

  0%|          | 0/3059 [00:00<?, ?it/s]/home/dhvanil21040/OpenPCDet/pcdet/models/model_utils/centernet_utils.py:261: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_classes = (topk_ind // K).int()
100%|██████████| 3059/3059 [12:45<00:00,  4.00it/s]
